In [ ]:
# Module1:  getting live ohlcv data 

# paral
import requests
import pandas as pd
from IPython.display import display
import websockets
import asyncio
import json
import talib as ta
symbol='BTCUSDT'
interval= '1d'

API_KEY = "e48e958cfca1c98466625133b3506ac6f9f700332b2cd7c44acace3ce2b5b2f5"
API_SECRET = "5f2b1ab22ce37425bc25df77f9a25bc10d1a7886ccf9e58c95a228309f7c0832"
url=f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit=5000'
data = requests.get(url=url)
data = data.json()


class PlaceOrder:
    def __init__(self):
        self.API_KEY="e48e958cfca1c98466625133b3506ac6f9f700332b2cd7c44acace3ce2b5b2f5"
        self.API_SECRET="5f2b1ab22ce37425bc25df77f9a25bc10d1a7886ccf9e58c95a228309f7c0832"
        self.BASE_URL = 'https://testnet.binance.vision'
        self.endpoint = '/api/v3/order'
    def buy(self):
        
        response = requests.get(BASE_URL + '/api/v3/time')
        server_time = response.json()['serverTime']
        self.params = {
        'symbol': 'BTCUSDT',
        'side': 'BUY',
        'type': 'MARKET',
        'quantity': 0.001,
        'recvWindow': 5000,  # Time for the request to be valid in milliseconds
        'timestamp': server_time, 
            }



        selfparams['signature'] = self.create_signature(self.params, self.API_SECRET)


        headers = {
        'X-MBX-APIKEY': self.API_KEY
        }

        response = requests.post(self.BASE_URL + self.endpoint, headers=headers, params=self.params)
        print(response.json())
        

    def sell(self):
        response = requests.get(BASE_URL + '/api/v3/time')
        server_time = response.json()['serverTime']
        self.params = {
        'symbol': 'BTCUSDT',
        'side': 'SELL',
        'type': 'MARKET',
        'quantity': 0.001,
        'recvWindow': 5000,  # Time for the request to be valid in milliseconds
        'timestamp': server_time, 
            }

        selfparams['signature'] = self.create_signature(self.params, self.API_SECRET)
        headers = {
        'X-MBX-APIKEY': self.API_KEY
        }

        response = requests.post(self.BASE_URL + self.endpoint, headers=headers, params=self.params)
        print(response.json())
        
    def create_signature(self,params, secret):
        query_string = '&'.join([f"{key}={params[key]}" for key in sorted(params)])
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
        




def sarCalculation(data):
    global pos
    pos='buy'
    sar = ta.SAR(data['High'], data['Low'], acceleration=0.05, maximum=0.5)
    if pos=='buy' and data['Close']>sar[-1]:
        ob = PlaceOrder()
        ob.buy()
    elif pos=='sell' and data['Close']<sar[-1]:
        ob = PlaceOrder()
        ob.sell()
    
    

columns = ['Open Time',"Open","High","Low","Close","Volume","Close Time",
           "Base asset volume","Number of trades","Taker buy volume","Taker buy base asset volume","Ignore"]
df = pd.DataFrame(data,columns=columns)

df["Open"] = df["Open"].astype(float)
df["Close"] = df["Close"].astype(float)
df["High"] = df["High"].astype(float)
df["Low"] = df["Low"].astype(float)
df["Volume"] = df["Volume"].astype(float)
df = df.set_index(pd.to_datetime(df["Open Time"],unit='ms'))

df = df.drop(['Base asset volume','Number of trades','Taker buy volume','Taker buy base asset volume','Ignore'],axis=1)


async def binance_kline(symbol,interval):
    global df
    url = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'
    async with websockets.connect(url) as wb:
        while True:
            message = await wb.recv()
            message = json.loads(message)
            
            if message.get("k").get("T")== df['Close Time'].iloc[-1]:
                # update last row
                df.iloc[-1] = [
                    float(message.get('k').get('t')),
                    float(message.get('k').get('o')),
                    float(message.get('k').get('h')),
                    float(message.get('k').get('l')),
                    float(message.get('k').get('c')),
                    float(message.get('k').get('v')),
                    float(message.get('k').get('T')),   
                ]
                
            else:
                # insert a new row
                d= pd.DataFrame({
                    "Open Time":float(message.get('k').get('t')),
                    "Open":float(message.get('k').get('o')),
                    "High":float(message.get('k').get('h')),
                    "Low":float(message.get('k').get('l')),
                    "Close":float(message.get('k').get('c')),
                    "Volume":float(message.get('k').get('v')),
                   "Close Time" :float(message.get('k').get('T')),
                },index=[pd.Timestamp.now()])
                
                df = pd.concat([df,d])
        sarCalculation(df)
            
            
display(df)
await binance_kline(symbol.lower(),interval)

In [ ]:
#!pip install websockets

In [ ]:
#list=[2,3,2,2,4,5,3,4,6,7,7,8,8,8]


In [ ]:
#!pip install Backtesting

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG
import talib as ta
testing_df=df.drop(['Close Time','Open Time'],axis=1)
#display(testing_df)
#print(testing_df.columns)
#print(type(testing_df))

class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 7)
        self.ma2 = self.I(SMA, price, 15)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


class RsiCross(Strategy):
    def init(self):
        price=self.data.close
        self.rsi= self.I(ta.RSI,price)
        self.overbough=70
    def next(self):
        if self.rsi <=self.oversell:
            self.buy()
        elif self.rsi <=self.overbought:
            self.sell()

class MARSI(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)
        self.overbought=70
        self.oversell=30
    def next(self):
        if crossover(self.ma1, self.ma2) or self.rsi<=self.oversell:
            self.buy()
        elif crossover(self.ma2, self.ma1) or self.rsi>=self.overbought:
            self.sell()


#SAR

class SARCross(Strategy):
    def init(self):
        self.high=self.data.High
        self.low=self.data.Low
        self.acceleration=0.04
        self.maximum=0.4
        self.sar=self.I(ta.SAR,self.high,self.low,acceleration=self.acceleration,maximum=self.maximum)
    def next(self):
        if crossover(self.data.Close,self.sar):
            self.buy()
        if crossover(self.sar,self.data.Close):
            self.sell()
        





bt = Backtest(testing_df, SARCross, commission=.002,
              exclusive_orders=True,cash=1000000)
#print(type(GOOG))
#print(GOOG.columns)
display(testing_df)
stats = bt.run()
print(stats)
bt.plot()

In [ ]:
# task-> test the strategies of backtesting by hitting again and again (changing symbols)
# changing moving averages
# find new strategies to apply in model 

In [ ]:
#import talib as ta

In [ ]:
import time
import hashlib
import hmac
import requests 

API_KEY="e48e958cfca1c98466625133b3506ac6f9f700332b2cd7c44acace3ce2b5b2f5"
API_SECRET="5f2b1ab22ce37425bc25df77f9a25bc10d1a7886ccf9e58c95a228309f7c0832"
BASE_URL="https://testnet.binance.vision"

response= requests.get(BASE_URL +'/api/v3/time')
server_time=response.json()["serverTime"]
print(server_time)

#Endpoint for placing order 
endpoint='/api/v3/order'

params = {
    'symbol': 'BTCUSDT',
    'side': 'BUY',
    'type': 'MARKET',
    'quantity': 0.001,
    'recvWindow': 5000,  # Time for the request to be valid in milliseconds
    'timestamp': server_time, 
}

def create_signature(params,secret):
    query_string= '&'.join([f"{key}={params[key]}" for key in sorted(params)])
    return hmac.new(secret.encode('utf-8'),query_string.encode('utf-8'),hashlib.sha256).hexdigest()
params['signature']=create_signature(params,API_SECRET)

headers={
    'X-MBX-APIKEY':API_KEY
}
response=requests.post(BASE_URL + endpoint, headers=headers, params=params)
print(response.json())
